# <span style="color:rgb(213,80,0)">Análisis de códigos</span>
Por Alejandro Islas Escobar
# Análisis scara_tray_line_py.py

Este código implementa un nodo de ROS 2 en Python que:
- Publica trayectorias articulares para un robot SCARA.
- Genera un movimiento **en línea recta** del efector final.
- Calcula los ángulos articulares mediante **cinemática inversa**.


En esta primer sección se realiza la importación de librerías que se usan en el código.

- rclpy: Librería de Python para trabajar con ROS 2.
- Node: Clase base para crear nodos ROS.
- JointTrajectory / JointTrajectoryPoint: Mensajes estándar para controlar robots con varios grados de libertad.
- Duration: Especifica tiempos en los mensajes de trayectoria.
- math: Se usa para el cálculo trigonométrico del modelo cinemático inverso.

In [ ]:
#!/usr/bin/env python3

import rclpy
from rclpy.node import Node
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from builtin_interfaces.msg import Duration

import time
from math import cos, sin, acos, asin, atan2, sqrt

Posteriormente iniciamos con la definición del nodo, el cual agrega el nombre las articulaciones del SCARA, la interpolación de ejecución y el tiempo en que debe recorrer la trayectoria.

In [ ]:
class ScaraTrayLineNode(Node):
    def __init__(self):
        super().__init__("scara_tray_line_node")
        topic_name = "/scara_trajectory_controller/joint_trajectory"
        self.joints_ = ['link_1_joint', 'link_2_joint', 'link_3_joint']
        self.lamda_ = 0
        self.Tiempo_ejec_ = 10

Luego se define un publicador que manda un mensaje del tipo JointTrajectory al tópico especificado. Con un timer ejecuta la función cada segundo y al momento de ejecutarse manda un mensaje indicando que el nodo está activo.

In [ ]:
        self.scara_tray_pub_ = self.create_publisher(
            JointTrajectory, topic_name, 10)
        self.tray_timer_ = self.create_timer(
            1, self.trayectory_cbck)
        self.get_logger().info(
            'Scara activo, trayectoria linea recta')


Posteriormente se define la función que genera un punto de una trayectoria a cada segundo. Este punto se envía al mensaje JointTrajectory que publicará el nodo.

In [ ]:
    def trayectory_cbck(self):
        trayectory_msg = JointTrajectory()
        trayectory_msg.joint_names = self.joints_
        point = JointTrajectoryPoint()

Lo primero que se realiza es definir los puntos iniciales y los puntos finales del efector final, esta función se ejecuta mientras la interpolación sea menor al tiempo para recorrer la trayectoria. Por cada valor lamda, se interpola un punto entre la posición inicial y la final.

In [ ]:
        if self.lamda_ <= self.Tiempo_ejec_:
            #Posición inicial
            x_1 = 0.1
            y_1 = 0.6
            theta_1 = 0
            #Posición final
            x_2 = 0.3
            y_2 = -0.6
            theta_2 = 1.57
            #Interpolación entre puntos
            solucion = invk_sol(self.lamda_, x_1, y_1,theta_1, x_2, y_2,theta_2)

Gracias a la interpolación, tenemos la posición en la que debería estar el efector final en cada segundo. Con esta información implementamos la cinématica inversa con la instrucción **point.positions** para obtener los valores (θ₁, θ₂, θ₃) de cada actuador y actualiza estos valores al publicador. Finalmente, el nodo publica el resultado y envía un mensaje al usuario de la posición actual.

In [ ]:
            point.positions = solucion
            point.time_from_start = Duration(sec=1)
            trayectory_msg.points.append(point)
            self.scara_tray_pub_.publish(trayectory_msg)
            self.get_logger().info("Postura actual {}".format(solucion))
            time.sleep(2)
            self.lamda_ += 1

En cuanto el valor de interpolación supera el tiempo para cumplir la trayectoria, detiene por completo el movimiento y los actuadores pasan a su posición de reposo.

In [ ]:
        elif self.lamda_ > 10:
            link_1_joint = 0
            link_2_joint = 0
            link_3_joint = 0
            return [float(link_1_joint), float(link_2_joint), float(link_3_joint)]

Para la solución debemos establecer la cinemática inversa del robot, para que al conocer la posición del efector final podamos ajustar los actuadores. Se define una función que primero incluye los parámetros del robot, como la longitud de sus eslabones y el tiempo en que debe recorrer la trayectoria. Esta función necesita recibir los valores del punto inicial y del punto final, con los cuales hace una interpolación lineal de cada punto entre estos dos puntos.

In [ ]:
def invk_sol(param,x_in, y_in, theta_in, x_fin, y_fin, theta_fin):
    Tiempo_ejec_ = 10
    L_1 = 0.5
    L_2 = 0.5
    L_3 = 0.3
    x_P = x_in + (param/Tiempo_ejec_)*(x_fin - x_in)
    y_P = y_in + (param/Tiempo_ejec_)*(y_fin - y_in)
    theta_P = theta_in + (param/Tiempo_ejec_)*(theta_fin - theta_in)

Con los datos de la posición, primero se declara la posición del último eslabón o muñeca y a partir de ello se obtiene el ángulo del resto de brazos (θ₁ y θ₂) utilizando la ley de cosenos.

In [ ]:
    x_3 = x_P - L_3*cos(theta_P)
    y_3 = y_P - L_3*sin(theta_P)
    theta_2 = acos((pow(x_3, 2)+pow(y_3,2)-pow(L_1, 2)-pow(L_2, 2))/(2*L_1*L_2))
    beta = atan2(y_3, x_3)
    psi = acos((pow(x_3, 2)+pow(y_3,2)+pow(L_1, 2)-pow(L_2, 2))/(2*L_1*sqrt(pow(x_3, 2)+pow(y_3,2))))
    theta_1 = beta - psi
    

Para el ángulo del último actuador, simplemente se realiza la diferencia del punto deseado menos los ángulos del resto de actuadores.

In [ ]:
theta_3 = theta_P -theta_1 -theta_2
    return [float(theta_1), float(theta_2), float(theta_3)]

Finalmente, se inicia el entorno de ROS donde el nodo queda en ejecución, con la posibilidad de detenerlo en cualquier momento.

In [ ]:
def main(args=None):
    rclpy.init(args=args)
    node = ScaraTrayLineNode()
    rclpy.spin(node)
    rclpy.shutdown()

if __name__ == "__main__":
    main()

# Análisis dofbot_sequence_py.py
Este segundo script controla un robot DOFBOT de 5 grados de libertad con su gripper o pinza. Además se simula la **cinemática inversa** y las acciones secuenciales de apertura, cierre y movimientos. El nodo publica mensajes en dos tópicos:

- /dofbot_trajectory_controller/joint_trajectory → movimiento del brazo.
- /dofbot_gripper_controller/joint_trajectory → apertura y cierre del gripper.

Igualmente, la primer sección del código es la importación de librerias que se usarán en el programa.
- rclpy: Librería de Python para trabajar con ROS 2.
- Node: Clase base para crear nodos ROS.
- JointTrajectory / JointTrajectoryPoint: Mensajes estándar para controlar robots con varios grados de libertad.
- Duration: Especifica tiempos en los mensajes de trayectoria.
- math: Se usa para el cálculo trigonométrico del modelo cinemático inverso.

In [ ]:
#!/usr/bin/env python3
import rclpy
from rclpy.node import Node
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from builtin_interfaces.msg import Duration
import time
from math import cos, sin, acos, asin, atan2, sqrt

Posteriormente se inicia la declaración del nodo de control. Aqui se declaran los tópicos para enviar trayectorias del brazo y del gripper.

In [ ]:
class DofbotControlNode(Node):
    def __init__(self):
        super().__init__("dofbot_tray_control_node")
        self.lamda_ = 0
        topic_dofbot_ = "/dofbot_trajectory_controller/joint_trajectory"
        topic_gripper_ = "/dofbot_gripper_controller/joint_trajectory"
        
        #Crea el publicador del brazo con 5 articulaciones
        self.dofbot_publisher_ = self.create_publisher(JointTrajectory, topic_dofbot_, 10)
        self.dofbot_joints_ = ['arm_joint_01', 'arm_joint_02','arm_joint_03', 'arm_joint_04', 'arm_joint_05']
        
        #Crea el publicador del gripper con 6 servomotores
        self.gripper_publisher_ = self.create_publisher(JointTrajectory, topic_gripper_, 10)
        self.gripper_joints_ = ['grip_joint', 'rfinger_joint_01', 'rfinger_joint_02', 'lfinger_grip_joint_01','lfinger_grip_joint_02', 'lfinger_grip_joint_03']
        

Al igual que el anterior programa, se establece un temporizador para que el robot avance cada 0.5 segundos.

In [ ]:
        self.timer_ = self.create_timer(0.5, self.timer_callback)
        self.get_logger().info('Nodo de control del dofbot en funcionamiento')

A continuación, se define la función **timer_callback**, la cual controla la secuencia del robot, utilizando un incremento en *lamda* y realizando una acción diferente para cada valor de la variable. 

In [ ]:
    def timer_callback(self):
        dofbot_msg = JointTrajectory()
        dofbot_msg.joint_names = self.dofbot_joints_
        dofbot_point = JointTrajectoryPoint()

        gripper_msg = JointTrajectory()
        gripper_msg.joint_names = self.gripper_joints_
        gripper_point = JointTrajectoryPoint()

Cuando el valor de lamda es 0, o bien el robot está iniciando, se usa la función **gripper_state** para mandar los valores articulares necesarios para tener un el valor que se declara de 1.57, el cual significa que el gripper está abierto a 90°.

In [ ]:
        if self.lamda_ == 0:
            # Open gripper
            gstate = 1.57
            gripper_st = gripper_state(gstate)
            gripper_point.positions = gripper_st
            gripper_point._time_from_start = Duration(sec=1)
            gripper_msg.points.append(gripper_point)
            self.gripper_publisher_.publish(gripper_msg)
            self.get_logger().info('Gripper open')
            self.get_logger().info('poture {}'.format(gripper_st))
            time.sleep(5)
            self.lamda_ +=1

Al terminar el paso anterior, a lamda se le aumenta en 1 su valor, lo que lo hace pasar a la siguiente etapa, en la que usando  el mismo comando de **gripper_state** se cierra completamente la pinza del robot.

In [ ]:
        elif self.lamda_ == 1:
            # Close gripper
            gstate_2 = 0
            gripper_st = gripper_state(gstate_2)
            gripper_point.positions = gripper_st
            gripper_point._time_from_start = Duration(sec=1)
            gripper_msg.points.append(gripper_point)
            self.gripper_publisher_.publish(gripper_msg)
            self.get_logger().info('Gripper close')
            self.get_logger().info('poture {}'.format(gripper_st))
            time.sleep(5)
            self.lamda_ +=1

Cuando lamda aumenta al tercer paso, solo hay una repetición de la apertura de la pinza.

In [ ]:
        elif self.lamda_ == 2:
            # Open gripper
            gstate = 1.57
            gripper_st = gripper_state(gstate)
            gripper_point.positions = gripper_st
            gripper_point._time_from_start = Duration(sec=1)
            gripper_msg.points.append(gripper_point)
            self.gripper_publisher_.publish(gripper_msg)
            self.get_logger().info('Gripper open')
            self.get_logger().info('poture {}'.format(gripper_st))
            time.sleep(5)
            self.lamda_ +=1

El siguiente paso a cumplir es un movimiento a unas coordenadas especifícas, por lo que se usa **positions** y **dofbot_ink** para obtener la cinemática inversa y poder mandar al publicador la configuración de los ángulos de nuestros actuadores.

In [ ]:
        elif self.lamda_ == 3:
            #Primera postura
            x_1 = 0.2
            y_1 = 0.0
            z_1 = 0.05
            theta_p_1 = 3.1416*(3/4)
            theta_g_1 = 0.0
            solution_pos = dofbot_ink(x_1, y_1, z_1, theta_p_1, theta_g_1)
            dofbot_point.positions = solution_pos
            dofbot_point.time_from_start = Duration(sec=2)
            dofbot_msg.points.append(dofbot_point)
            self.dofbot_publisher_.publish(dofbot_msg)
            self.get_logger().info('poture {}'.format(solution_pos))
            time.sleep(15)
            self.lamda_ += 1

El paso 4 vuelva a cerrar la pinza, mientras que los paso 5, 6 y 7 siguen la estructura del paso 3, donde se declaran las posiciones del robot y se usa la función para el cálculo de la cinemática inversa para ajustar los ángulos en las articulaciones.

In [ ]:
        elif self.lamda_ == 4:
            # Close gripper
            gstate_2 = 0
            gripper_st = gripper_state(gstate_2)
            gripper_point.positions = gripper_st
            gripper_point._time_from_start = Duration(sec=2)
            gripper_msg.points.append(gripper_point)
            self.gripper_publisher_.publish(gripper_msg)
            self.get_logger().info('Gripper close')
            self.get_logger().info('poture {}'.format(gripper_st))
            time.sleep(10)
            self.lamda_ +=1

        elif self.lamda_ == 5:
            # Segunda postura
            x_2 = 0.15
            y_2 = 0.0
            z_2 = 0.11
            theta_p_2 = 3.1416*(3/4)
            theta_g_2 = 0 
            solution_pos = dofbot_ink(x_2, y_2, z_2, theta_p_2, theta_g_2)
            dofbot_point.positions = solution_pos
            dofbot_point.time_from_start = Duration(sec=2)
            dofbot_msg.points.append(dofbot_point)
            self.dofbot_publisher_.publish(dofbot_msg)
            self.get_logger().info('poture {}'.format(solution_pos))
            time.sleep(15)
            self.lamda_ += 1

        elif self.lamda_ == 6:
            # tercer postura
            x_3 = 0.15
            y_3 = 0.15
            z_3 = 0.11
            theta_p_3 = 3.1416*(3/4) 
            theta_g_3 = 0 
            solution_pos = dofbot_ink(x_3, y_3, z_3, theta_p_3, theta_g_3)
            dofbot_point.positions = solution_pos
            dofbot_point.time_from_start = Duration(sec=2)
            dofbot_msg.points.append(dofbot_point)
            self.dofbot_publisher_.publish(dofbot_msg)
            self.get_logger().info('poture {}'.format(solution_pos))
            time.sleep(15)
            self.lamda_ += 1

        elif self.lamda_ == 7:
            # cuarta postura
            x_3 = 0.15
            y_3 = 0.15
            z_3 = 0.05
            theta_p_3 = 3.1416*(3/4) 
            theta_g_3 = 0 
            solution_pos = dofbot_ink(x_3, y_3, z_3, theta_p_3, theta_g_3)
            dofbot_point.positions = solution_pos
            dofbot_point.time_from_start = Duration(sec=2)
            dofbot_msg.points.append(dofbot_point)
            self.dofbot_publisher_.publish(dofbot_msg)
            self.get_logger().info('poture {}'.format(solution_pos))
            time.sleep(15)
            self.lamda_ += 1

Una vez alcanzada la cuarta posición o postura, se vuelve a abrir la pinza en el paso 9.

In [ ]:
        elif self.lamda_ == 8:
            # Open gripper
            gstate = 1.57
            gripper_st = gripper_state(gstate)
            gripper_point.positions = gripper_st
            gripper_point._time_from_start = Duration(sec=2)
            gripper_msg.points.append(gripper_point)
            self.gripper_publisher_.publish(gripper_msg)
            self.get_logger().info('Gripper open')
            self.get_logger().info('poture {}'.format(gripper_st))
            time.sleep(10)
            self.lamda_ +=1

Como paso final, el robot regresa a su posición inicial, donde los ángulos de las articulaciones pasan a ser 0.

In [ ]:
        elif self.lamda_ == 9:
            solution_pos = [ float(0.0), float(0.0), float(0.0), float(0.0), float(0.0)]
            dofbot_point.positions = solution_pos
            dofbot_point.time_from_start = Duration(sec=2)
            dofbot_msg.points.append(dofbot_point)
            self.dofbot_publisher_.publish(dofbot_msg)
            self.get_logger().info('poture {}'.format(solution_pos))
            time.sleep(10)

Como se menciona anteriormente, **dofbot_ink** es la función que tiene el cálculo de la cinemática inversa para obtener la posición de los actuadores. Tiene como parámetros de entrada las coordenadas y ángulos del punto final, además de tener la configuración de las longitudes de los eslabones del motor. Con los parámetros se usa trigonometría y el teorema del coseno para obtener los ángulos de las articulaciones según la posición deseada (x_P, y_P, z_P) y la orientación theta_1_P. Todo de la misma forma que el primer script.

In [ ]:
def dofbot_ink(x_P, y_P, z_P, theta_1_P, theta_g):
    # Parametros
    z_0_1 = 0.105
    L_1 = 0.084
    L_2 = 0.084
    L_3 = 0.115

    theta_1 = atan2(y_P, x_P)
    aux_x = sqrt(pow(x_P, 2) + pow(y_P, 2)) - L_3*sin(theta_1_P)
    aux_z = z_P - z_0_1 -L_3*cos(theta_1_P)
    norm_4_P = sqrt(pow(aux_z, 2)+pow(aux_x, 2))
    epsilon = acos(aux_z/norm_4_P)
    alpha = acos((pow(L_1, 2)+pow(norm_4_P, 2)-pow(L_2, 2))/(2*L_1*norm_4_P))
    theta_2 = epsilon - alpha
    theta_3 = 3.1416 - asin((sin(alpha)*sqrt(pow(aux_x, 2) + pow(aux_z, 2)))/(L_2))
    theta_4 = theta_1_P - theta_2 - theta_3
    theta_5 = theta_g
    return [ float(theta_1), float(-theta_2), float(-theta_3), float(-theta_4), float(theta_5)]

La definición de posición del gripper únicamente indica la configuración de los servomotores que componen el gripper para que mantengan la pinza cerrada o abierta segun la geometría en la que están acomodados.

In [ ]:
def gripper_state(theta):
    
    return [float(-theta), float(theta), float(-theta), float(theta), float(-theta), float(theta)]

Finalmente, se inicia el entorno en ROS, se inicializa el nodo y se mantiene en ejecución hasta que se detenga el programa, lo cual se establece con la función **shutdown**.

In [ ]:
def main(args=None):
    rclpy.init(args=args)
    node = DofbotControlNode()
    rclpy.spin(node)
    rclpy.shutdown()

if __name__ == "__main__":
    main()